# M&M model VEM updates

In [1]:
dat = readRDS('/home/gaow/Documents/GTExV8/Thyroid.Lung.FMO2.filled.rds')
attach(dat)

In [2]:
%get X Y --from R

Loading required package: feather


In [3]:
from libgaow.regression_data import RegressionData
import numpy as np

In [11]:
class MNMASH(RegressionData):
    def __init__(self, X=None, Y=None, Z=None, B=None, S=None, V=None):
        RegressionData.__init__(self, X, Y, Z, B, S)
        self.V = V
        self.alpha0 = None
        self.mu0 = None
        self.elbo = None
    
    def set_prior(self):
        pass

    def fit(self, niter=100, L=5, calc_elbo=False):
        if self.V is None:
            self.V = np.cov(self.Y, rowvar=False)
        pass

    def _calc_update(self):
        pass

    def _calc_single_snp(self):
        pass

    def _calc_elbo(self):
        pass

In [12]:
class ELBOMNM:
    def __init__(self, data):
        self.data = data